# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
weather_check = pd.read_csv("../WeatherPy/weather_check.csv",index_col=[0])
weather_check.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,82.40,78,75,2.30,RE,1610594927
1,Punta Arenas,-53.1500,-70.9167,53.60,71,90,13.80,CL,1610594565
2,Lashio,22.9333,97.7500,63.46,66,14,0.45,MM,1610594651
3,George Town,5.4112,100.3354,82.40,69,20,8.05,MY,1610594676
4,Genhe,50.7833,121.5167,-14.58,82,68,2.77,CN,1610594928


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#from config import gkey
gmaps.configure(api_key=g_key)

In [24]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_check[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#weather_check = census_data_complete.dropna()
Humidity = weather_check["Humidity"].astype(float)

In [25]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
##Narrow down the DataFrame to find your ideal weather condition. For example: A max temperature lower than 80 degrees 
##but higher than 70. Wind speed less than 10 mph. Zero cloudiness. Drop any rows that don't contain all three 
##conditions. You want to be sure the weather is ideal. Note: Feel free to adjust to your specifications but 
##be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal = weather_check.loc[(weather_check["Max Temp"] > 85) & (weather_check["Max Temp"] < 95) & (weather_check['Cloudiness'] < 60) & (weather_check['Wind Speed'] < 20)] 
ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Kingaroy,-26.5333,151.8333,86.00,39,40,6.91,AU,1610594930
54,Narrabri,-30.3167,149.7833,93.20,26,0,5.75,AU,1610594994
99,Port Macquarie,-31.4333,152.9167,91.00,48,0,14.50,AU,1610594981
108,Neiafu,-18.6500,-173.9833,86.00,74,40,10.36,TO,1610595004
163,Carnarvon,-24.8667,113.6333,86.00,70,45,14.97,AU,1610595052
196,Tautira,-17.7333,-149.1500,87.80,62,40,4.61,PF,1610595141
240,Biloela,-24.4167,150.5000,91.42,27,0,13.78,AU,1610595208
288,Flinders,-34.5833,150.8552,91.00,62,8,4.00,AU,1610595274
310,Bundaberg,-24.8500,152.3500,88.00,45,32,14.97,AU,1610595081
483,Padang,-0.9492,100.3543,86.00,62,20,1.14,ID,1610595545


In [ ]:
##Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
##Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [32]:
ideal.shape

(11, 9)

In [35]:
ideal['target_coordinates'] = ideal[["Lat","Lng"]].apply(lambda x: f"{x[0]}, {x[1]}", axis=1)

##Store into variable named hotel_df.

ideal

<ipython-input-35-2239e6a9beef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal['target_coordinates'] = ideal[["Lat","Lng"]].apply(lambda x: f"{x[0]}, {x[1]}", axis=1)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,target_coordinates
27,Kingaroy,-26.5333,151.8333,86.00,39,40,6.91,AU,1610594930,"-26.5333, 151.8333"
54,Narrabri,-30.3167,149.7833,93.20,26,0,5.75,AU,1610594994,"-30.3167, 149.7833"
99,Port Macquarie,-31.4333,152.9167,91.00,48,0,14.50,AU,1610594981,"-31.4333, 152.9167"
108,Neiafu,-18.6500,-173.9833,86.00,74,40,10.36,TO,1610595004,"-18.65, -173.9833"
163,Carnarvon,-24.8667,113.6333,86.00,70,45,14.97,AU,1610595052,"-24.8667, 113.6333"
196,Tautira,-17.7333,-149.1500,87.80,62,40,4.61,PF,1610595141,"-17.7333, -149.15"
240,Biloela,-24.4167,150.5000,91.42,27,0,13.78,AU,1610595208,"-24.4167, 150.5"
288,Flinders,-34.5833,150.8552,91.00,62,8,4.00,AU,1610595274,"-34.5833, 150.8552"
310,Bundaberg,-24.8500,152.3500,88.00,45,32,14.97,AU,1610595081,"-24.85, 152.35"
483,Padang,-0.9492,100.3543,86.00,62,20,1.14,ID,1610595545,"-0.9492, 100.3543"


In [37]:
ideal["Hotel Name"]=""
ideal

<ipython-input-37-b42a0c320128>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal["Hotel Name"]=""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,target_coordinates,hotel,Hotel Name
27,Kingaroy,-26.5333,151.8333,86.00,39,40,6.91,AU,1610594930,"-26.5333, 151.8333",,
54,Narrabri,-30.3167,149.7833,93.20,26,0,5.75,AU,1610594994,"-30.3167, 149.7833",,
99,Port Macquarie,-31.4333,152.9167,91.00,48,0,14.50,AU,1610594981,"-31.4333, 152.9167",,
108,Neiafu,-18.6500,-173.9833,86.00,74,40,10.36,TO,1610595004,"-18.65, -173.9833",,
163,Carnarvon,-24.8667,113.6333,86.00,70,45,14.97,AU,1610595052,"-24.8667, 113.6333",,
196,Tautira,-17.7333,-149.1500,87.80,62,40,4.61,PF,1610595141,"-17.7333, -149.15",,
240,Biloela,-24.4167,150.5000,91.42,27,0,13.78,AU,1610595208,"-24.4167, 150.5",,
288,Flinders,-34.5833,150.8552,91.00,62,8,4.00,AU,1610595274,"-34.5833, 150.8552",,
310,Bundaberg,-24.8500,152.3500,88.00,45,32,14.97,AU,1610595081,"-24.85, 152.35",,
483,Padang,-0.9492,100.3543,86.00,62,20,1.14,ID,1610595545,"-0.9492, 100.3543",,


In [48]:
for idx,row in ideal.iterrows():
    print(row.target_coordinates)
    # geocoordinates
    target_coordinates = row.target_coordinates
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #insert Hotel name into "Hotel Name" column
    ideal.loc[idx,"Hotel Name"]=response.json()["results"][1]["name"]
  

-26.5333, 151.8333


C:\Users\grety\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


-30.3167, 149.7833
-31.4333, 152.9167
-18.65, -173.9833
-24.8667, 113.6333
-17.7333, -149.15
-24.4167, 150.5
-34.5833, 150.8552
-24.85, 152.35
-0.9492, 100.3543
-20.0167, 148.2333


In [49]:
ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,target_coordinates,hotel,Hotel Name
27,Kingaroy,-26.5333,151.8333,86.00,39,40,6.91,AU,1610594930,"-26.5333, 151.8333",,Ascot Lodge Motel Kingaroy
54,Narrabri,-30.3167,149.7833,93.20,26,0,5.75,AU,1610594994,"-30.3167, 149.7833",,Crossroads Hotel
99,Port Macquarie,-31.4333,152.9167,91.00,48,0,14.50,AU,1610594981,"-31.4333, 152.9167",,Rydges Port Macquarie
108,Neiafu,-18.6500,-173.9833,86.00,74,40,10.36,TO,1610595004,"-18.65, -173.9833",,Mystic Sands
163,Carnarvon,-24.8667,113.6333,86.00,70,45,14.97,AU,1610595052,"-24.8667, 113.6333",,Hospitality Carnarvon
196,Tautira,-17.7333,-149.1500,87.80,62,40,4.61,PF,1610595141,"-17.7333, -149.15",,Pension Oaoa
240,Biloela,-24.4167,150.5000,91.42,27,0,13.78,AU,1610595208,"-24.4167, 150.5",,Silo Motor Inn
288,Flinders,-34.5833,150.8552,91.00,62,8,4.00,AU,1610595274,"-34.5833, 150.8552",,Shellharbour Resort & Conference Centre
310,Bundaberg,-24.8500,152.3500,88.00,45,32,14.97,AU,1610595081,"-24.85, 152.35",,Takalvan Motel
483,Padang,-0.9492,100.3543,86.00,62,20,1.14,ID,1610595545,"-0.9492, 100.3543",,Plaza Andalas


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal.iterrows()]
locations = ideal[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [77]:
# Create hotel symbol layer 
hotel_layer = gmaps.marker_layer(
    locations, 
    
    info_box_content=hotel_info
)


fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))